In [1]:
import flash
from flash.image import ObjectDetectionData, ObjectDetector

In [5]:
datamodule = ObjectDetectionData.from_coco(
    train_folder="coco128/images/train2017/",
    train_ann_file="coco128/annotations/instances_train2017.json",
    val_split=0.1,
    transform_kwargs={"image_size": 512},
    batch_size=64,
)

  0%|          | 0/929 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/126 [00:00<?, ?it/s]

AUTOFIX-SUCCESS - (record_id: 10) - Clipping bbox xmax from 640.00032 to image width 640 (Before: <BBox (xmin:617.67008, ymin:237.43003499999998, xmax:640.00032, ymax:250.230057)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 11) - Clipping bbox ymax from 640.0003200000001 to image height 640 (Before: <BBox (xmin:50.450050000000005, ymin:72.07008000000002, xmax:283.96012399999995, ymax:640.0003200000001)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 22) - Clipping bbox ymax from 480.00023999999996 to image height 480 (Before: <BBox (xmin:294.93984, ymin:367.89, xmax:484.63008, ymax:480.00023999999996)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 22) - Clipping bbox ymin from -0.00023999999999091415 to 0 (Before: (<BBox (xmin:208.18016, ymin:-0.00023999999999091415, xmax:640.00032, ymax:474.60983999999996)>)) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 22) - Cl

AUTOFIX-SUCCESS - (record_id: 47) - Clipping bbox xmax from 428.000214 to image width 428 (Before: <BBox (xmin:148.650178, ymin:67.01983999999999, xmax:428.000214, ymax:356.0496)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 50) - Clipping bbox xmin from -0.00031999999999499096 to 0 (Before: <BBox (xmin:-0.00031999999999499096, ymin:236.44980899999996, xmax:119.65984, ymax:421.209843)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 53) - Clipping bbox xmin from -0.00031999999999854367 to 0 (Before: <BBox (xmin:-0.00031999999999854367, ymin:323.200143, xmax:63.339839999999995, ymax:407.60010900000003)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 58) - Clipping bbox xmax from 640.0003200000001 to image width 640 (Before: <BBox (xmin:587.38016, ymin:0.0, xmax:640.0003200000001, ymax:149.4)>) | icevision.utils.logger_utils:autofix_log:17
AUTOFIX-SUCCESS - (record_id: 58) - Clipping bbox xmin from

In [6]:
heads = ObjectDetector.available_heads()

# print the heads
# print('HEADS',*heads, sep='\n')
# print the backbones
for i in heads:
    print(i)
    backbones = ObjectDetector.available_backbones(i)
    print('BACKBONES', *backbones, sep='\n')

efficientdet
BACKBONES
d0
d1
d2
d3
d4
d5
d6
d7
d7x
tf_d0
tf_d0_ap
tf_d1
tf_d1_ap
tf_d2
tf_d2_ap
tf_d3
tf_d3_ap
tf_d4
tf_d4_ap
tf_d5
tf_d5_ap
tf_d6
tf_d7
tf_d7x
tf_lite0
tf_lite1
tf_lite2
tf_lite3
faster_rcnn
BACKBONES
resnet101_fpn
resnet152_fpn
resnet18_fpn
resnet34_fpn
resnet50_fpn
resnext101_32x8d_fpn
resnext50_32x4d_fpn
wide_resnet101_2_fpn
wide_resnet50_2_fpn
retinanet
BACKBONES
resnet101_fpn
resnet152_fpn
resnet18_fpn
resnet34_fpn
resnet50_fpn
resnext101_32x8d_fpn
resnext50_32x4d_fpn
wide_resnet101_2_fpn
wide_resnet50_2_fpn
yolov5
BACKBONES
extra_large
extra_large_p6
large
large_p6
medium
medium_p6
small
small_p6


In [7]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from icevision.all import *

metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

In [8]:
model = ObjectDetector(
    head='efficientdet',
    backbone='d0',
    num_classes=datamodule.num_classes,
    image_size=512,
    optimizer="Adam",
    learning_rate=1e-4,
    metrics=metrics,
)

Using 'efficientdet' provided by airctic/IceVision (https://github.com/airctic/icevision) and rwightman/efficientdet-pytorch (https://github.com/rwightman/efficientdet-pytorch).


In [9]:
trainer = flash.Trainer(max_epochs=3, accelerator='gpu', devices=1, precision=16)
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                            | Params
------------------------------------------------------------------
0 | train_metrics | ModuleDict                      | 0     
1 | val_metrics   | ModuleDict                      | 0     
2 | test_metrics  | ModuleDict                      | 0     
3 | adapter       | IceVisionObjectDetectionAdapter | 3.9 M 
------------------------------------------------------------------
88.0 K    Trainable params
3.8 M     Non-trainable params
3.9 M     Total params
7.756     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/demontego/miniconda3/envs/sbercluod/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/demontego/miniconda3/envs/sbercluod/lib/python3.9/site-packages/effdet/bench.py:45: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices_all = cls_topk_indices_all // num_classes
/home/demontego/miniconda3/envs/sbercluod/lib/python3.9/site-pa

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [10]:
trainer.save_checkpoint("object_detection_model_1epoch.pt")

In [118]:
datamodule = ObjectDetectionData.from_files(
    predict_files=[
        Path(flash.ASSETS_ROOT) / "fish.jpg",
        "coco128/images/train2017/000000000626.jpg",
        "coco128/images/train2017/000000000629.jpg",
    ],
    transform_kwargs={"image_size": 512},
    batch_size=4,
)
predictions = trainer.predict(model, datamodule=datamodule)
print(predictions)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 1it [00:00, ?it/s]

[[{<DataKeys.INPUT: 'input'>: {<DataKeys.METADATA: 'metadata'>: {'size': (224, 224), 'output_size': (512, 512), 'filepath': Path('/home/demontego/miniconda3/envs/sbercluod/lib/python3.9/site-packages/flash/assets/fish.jpg')}, <DataKeys.INPUT: 'input'>: Path('/home/demontego/miniconda3/envs/sbercluod/lib/python3.9/site-packages/flash/assets/fish.jpg'), <DataKeys.TARGET: 'target'>: {}}, <DataKeys.PREDS: 'preds'>: {'bboxes': [], 'labels': [], 'scores': []}, <DataKeys.METADATA: 'metadata'>: {'size': [224, 224], 'output_size': [512, 512], 'filepath': Path('/home/demontego/miniconda3/envs/sbercluod/lib/python3.9/site-packages/flash/assets/fish.jpg')}}, {<DataKeys.INPUT: 'input'>: {<DataKeys.METADATA: 'metadata'>: {'size': (480, 640), 'output_size': (512, 512), 'filepath': 'coco128/images/train2017/000000000626.jpg'}, <DataKeys.INPUT: 'input'>: 'coco128/images/train2017/000000000626.jpg', <DataKeys.TARGET: 'target'>: {}}, <DataKeys.PREDS: 'preds'>: {'bboxes': [{'xmin': 249.1985, 'ymin': 160.2

In [1]:
import base64
from pathlib import Path

import requests

with open("coco128/images/train2017/000000000626.jpg","rb") as f:
    imgstr = base64.b64encode(f.read()).decode("UTF-8")

body = {"session": "UUID", "payload": {'inputs': {'data': imgstr}}}
resp = requests.post("http://127.0.0.1:8000/predict", json=body)
print(resp)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe0802b1af0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [141]:
resp.json()

{'session': 'UUID',
 'result': {'outputs': [{'confidence': 0.8353,
    'label': '85',
    'points': [311, 120, 43, 46]},
   {'confidence': 0.8147, 'label': '85', 'points': [397, 129, 19, 45]}]}}